# Template for Solving Models

Every basic Gurobi optimization model can be constructed with the code template shown below.  This template is provided as a starting point for each problem formulations below. Feel free to use it for your work.

```python
import gurobipy as gp
from gurobipy import GRB

''' Import or define problem data '''

''' Create Gurobi model object '''
m = gp.Model('') # insert model name in quotes

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MAXIMIZE

''' Specify optimization parameter settings, if desired '''
# m.setParam('TimeLimit', 7200)

''' Create decision variables and update model'''
# Use m.addVar(), m.addVars() or m.addMVar() here
m.update()

''' Create objective function and update model '''
m.setObjective()
m.update()

''' Create constraints and update model '''
# Use m.addConstr(), m.addLConstr(), m.addConstrs(), or m.addMConstr() here
m.update()

''' Optimize model '''
m.optimize()

''' Print results '''

```

# Exploring the Galaxy Toys Example

We are now going to explore the Galaxy Toys example a bit more. Let's first see how to create and solve the model using Gurobi. Then, how can we get the sensitivity analysis information out?

In [ ]:
# import the two Gurobi parts we need
import gurobipy as gp
from gurobipy import GRB
# import pandas for help in functions
import pandas as pd

In [ ]:
# Define a function that will return sensitivity analysis for the variables
def sa_vars(the_vars):
    """
    This function takes the variables from a solved model
    and returns a pandas DataFrame where the index is the
    variable name and the columns are the (1) the resulting
    value of the variable, (2) the reduced cost, (3) the
    objective function coefficient, (4) the low end of the
    range of optimality, and (5) the high end of the range
    of optimality

    Parameters
    ===============
    the_vars: the variables of the solved model (e.g., the
              result of m.getVars())

    Returns
    ===============
    A pandas DataFrame object
    """
    sa = {}
    for v in the_vars:
        sa[v.VarName] = [v.X, v.RC, v.Obj, v.SAObjLow, v.SAObjUp]

    return pd.DataFrame.from_dict(sa,
                       orient='index',
                       columns=['final_value', 'reduced_cost', 'obj_coef', 'range_opt_low', 'range_opt_high'])

In [ ]:

def sa_constrs(the_constrs):
    """
    This function takes the constraints from a solved model
    and returns a pandas DataFrame where the index is the
    constraint name and the columns are the (1) indication 
    if the constraint is binding or non-binding, (2) the
    value of the constraint, (3) the RHS of the constraint,
    (4) the slack, (5) the shadow price, (6) the low end of
    the range of feasibility, and (5) the high end of the
    range of feasibility

    Parameters
    ===============
    the_constrs: the constraints of the solved model (e.g., the
              result of m.getConstrs())

    Returns
    ===============
    A pandas DataFrame object
    """
    sa = {}
    for c in the_constrs:
        binding = 'binding'
        if c.Slack > 0.00001:
            binding = 'non-binding'
        sa[c.constrName] = [binding, c.RHS-c.Slack, c.RHS, c.Slack, c.pi, c.SARHSLow, c.SARHSUp]

    return pd.DataFrame.from_dict(sa,
                        orient='index',
                        columns=['binding?', 'final_value', 'RHS', 'slack', 'shadow_price', 'range_feas_low', 'range_feas_high'])

## Very Simple Approach

We are going to start with the most basic and "simple" way to create and solve our small LP using Gurobi. We will do it steps, with each step in its own code cell. (You can, of course, put all these steps into a single code cell.) NOTE: We should probably put a `try` block around all of this code to catch any runtime errors. 

## Formulation 

Recall that our formulation looked like this:

| | | |
| --- | --- | --- |
| Let | | |
| $x_{s}$ | = | number of lots (dozens) of Space Rays to produce next week |
| $x_{p}$ | = | number of lots (dozens) of Phasers to produce next week |

| | | | | | | |
| --- | --- | --- | --- | --- | --- | --- |
| $\max$ | $8x_{s}$ | $+$ | $5x_{p}$ | | | |
| s.t. | $2x_{s}$ | $+$ | $1x_{p}$ | $\le$ | $1200$ | {plastic pounds} |
| | $3x_{s}$ | $+$ | $4x_{p}$ | $\le$ | $2400$ | {minutes of production} |
| | $1x_{s}$ | $+$ | $1x_{p}$ | $\le$ | $800$ | {overall production limit} |
| | $1x_{s}$ | $-$ | $1x_{p}$ | $\le$ | $450$ | {mix of products produced} |
| | $x_{s}$ | | | $\ge$ | $0$ | {non-negativity} |
| | | | $x_{p}$ | $\ge$ | $0$ | {non-negativity} |

In [ ]:
# Create the model object
m = gp.Model('galaxy_toys')

In [ ]:
# Specify how to optimize and time limit (seconds)
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit', 600)

# update the model
m.update()

In [ ]:
# Create decision variables
# We tell the solver that the variables are continuous,
#   their names, and their lower bounds
x_s = m.addVar(vtype=GRB.CONTINUOUS, name='x_s', lb=0.0)
x_p = m.addVar(vtype=GRB.CONTINUOUS, name='x_p', lb=0.0)

# update the model
m.update()

In [ ]:
# Add the objective function
m.setObjective(8 * x_s + 5 * x_p)
m.update()

In [ ]:
# Add the constraints
# We can simply write out the constraints for the first parameter
# The second parameter names the constraint
m.addConstr(2*x_s + x_p <= 1200, name='plastic')
m.addConstr(3*x_s + 4*x_p <= 2400, name='labor')
m.addConstr(x_s + x_p <= 800, name='total_production')
m.addConstr(x_s - x_p <= 450, name='product_mix')
m.update()

In [ ]:
# This method is undocumented and deprecated
# BUT I think it still works
m.display()

In [ ]:
# solve
m.optimize()

### Getting the Results

We now want to get the results and print them in a little nicer format. 

In [ ]:
# Get the results out
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

### Sensitivity Analysis

We can easily get the reduced cost and the range of optimality for each variable. Similarly, we can extract the shadow price and the range feasibility for each constraint.

In [ ]:
# Get the reduced cost and range of optimality for each variable
for v in m.getVars():
    print(f'{v.VarName} has a reduced cost of {v.RC}')
    print(f'   and a range of optimality from {v.SAObjLow} to {v.SAObjUp}')

In [ ]:
# Get the shadow price and the range of feasibility for each constraint
for c in m.getConstrs():
    print(f'{c.constrName} has a shadow price of {c.pi}')
    print(f'   and a range of feasibility from {c.SARHSLow} to {c.SARHSUp}')
    print(f'   and a slack of {c.Slack}')
    print(f'   and a RHS of {c.RHS}')
    print(f'   and a final value of {m.getRow(c).getValue()}')

### Modularized Functionality

Up at the top of this notebook, I created two functions to make it easier to extract the sensitivity analysis for both the variables, `sa_vars()`, and the constraints, `sa_constrs()`. Let's use those to get the sensitivity information out. First, we'll examine the docstring on the function and then use it.

In [ ]:
help(sa_vars)

In [ ]:
# returns a DataFrame
# This will simply print it out
sa_vars(m.getVars())

### Range of Optimality

Suppose the objective function coefficient for space rays was really \$7. How does this change the solution? Can you answer that question using the sensitivity report? Do you have re-solve the problem witht the new data?

### Changing the Objective Function Coefficient for Space Rays

Let's go ahead and change the objective function coefficient from the original \\$8 to \\$7 and re-run the solver to see what we get.

In [ ]:
# Change the objective function
m.setObjective(7 * x_s + 5 * x_p)
m.update()
m.display()

In [ ]:
# Solve the updated model with $7
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

In [ ]:
# Look at sensitivity analysis for variables again
sa_vars(m.getVars())

### Reduced Costs

We want to change the objective function coefficient of Space Rays to \\$2 and re-solve the model. Note that \\$2 is outside the range of optimality, so we need to re-solve the model to find the optimal solution.

In [ ]:
# Change the objective function
m.setObjective(2 * x_s + 5 * x_p)
m.update()
m.display()

In [ ]:
# Solve the updated model with $2
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

In [ ]:
# Look at sensitivity analysis for variables again
sa_vars(m.getVars())

We see that the reduced cost of $x_{s}$ is $-1.75$. This implies that the profit coefficient for Space Rays would have to increase by \\$1.75, which would be \\$2 + \\$1.75 = \\$3.75, before it becomes economically feasible to produce Space Rays. (Thought exercise: does that number look familiar?)

Another way to think of the reduced cost is: 'What if I were forced to produce one unit (one lot or one dozen) of Space Rays? What would happen to my profit?' Let's add another constraint to the model that forces one unit of Space Rays production and see the results. 

In [ ]:
# Add a new constraint that says: x_s >= 1
# You could do this two ways:
# 1. as a structural constraint -- doing this one
# 2. by changing the lower bound on x_s
m.addConstr(x_s >= 1, name='force_space_ray_production')
m.update()
m.display()

In [ ]:
# Solve and display results
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

### Range of Feasibility

Suppose we had one additional pound of plastic that could be used for our production cycle. What does this additional resource do to our solution? Can you answer that with the sensitivity report?

First, let's go back to the original model. We need to reset the objective function coefficient for space rays and remove the constraint that forced the production of space rays. Then, let's change the right-hand-side of the plastic constraint and re-solve the problem.

In [ ]:
# See what the current model looks like
m.display()

#### A Quick Aside

If (and when) the method `.display()` goes away, you could do the following. Write the model to a temporary file in `.lp` format, then read it back in and print it. This approach is a **bad** idea for larger models. See below for an example. 

In [ ]:
# One work around is to write the .lp file 
# to disk and and read it back in and 
# print it out ... not recommended for large models
m.write('./tmp.lp')
with open('./tmp.lp') as f:
    print(f.read())

In [ ]:
# Let's change objective function back to the original
m.setObjective(8 * x_s + 5 * x_p)
# And delete the force space rays production constraint
# Get the constraint by name and remove it
c = m.getConstrByName('force_space_ray_production')
if c:  # Check if the constraint exists
    m.remove(c)

m.update()
m.display()

In [ ]:
# Solve the model
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

### Getting Sensitivity Information for Constraints

You can use the function defined at the top of this notebook, `sa_constrs()`, to retrieve the sensitivity analysis for the constraints. The returned value from the function is a pandas `DataFrame`. 

In [ ]:
# See the docstring for the function
help(sa_constrs)

In [ ]:
sa_constrs(m.getConstrs())

In [ ]:
# We can pull out a specific constraint by its name
# Let's get plastic and store it a Python variable named plastic
plastic = m.getConstrByName('plastic')
print(plastic.RHS)

In [ ]:
# Now update the RHS with one additional pound of plastic
plastic.RHS = 1201

# Update the model and solve it
m.update()
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

In [ ]:
# sensitivity analysis for variables
sa_vars(m.getVars())

In [ ]:
# sensitivity analysis for constraints
sa_constrs(m.getConstrs())

### Multiple Optimal Solutions

Suppose the model has multiple optimal solutions. Can we find them? Let's change the objective function coefficient of space rays to \\$3.75 and re-solve the model. (We also need to change the plastic constraint's RHS back to 1200.) 

In [ ]:
# See the current model
m.display()

In [ ]:
# reset the rhs of the plastic constraint to 1200
print(f'Current RHS of plastic constraint is {plastic.rhs}')

plastic.rhs = 1200

# update the model
m.update()

print(f'After changing RHS of back to original is {plastic.rhs}')

In [ ]:
# Change the objective function
m.setObjective(3.75 * x_s + 5 * x_p)
m.update()
m.display()

In [ ]:
# Solve
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

In [ ]:
sa_vars(m.getVars())

#### Multiple Optimal Solutions?

I am going to claim that this current model has multiple optimal solutions. One way to determine if that is true is to plug all of the corner points (we only had 5) into the objective function, $3.75x_{s} + 5x_{p}$, and see if any of the other corner points give us \\$3,000. (You can go back to where we visualized the model in a previous notebook and found all the corner points.)

1. (0, 0) = \\$0
2. (450, 0) = \\$1,687.50
3. (550, 100) = \\$2,562.50
4. (480, 240) = \\$3,000
5. (0, 600) = \\$3,000

So, yes, there are multiple optimal solutions to this model. Another clue that an alternate optimal solution exists can be seen in the sensitivity information: the objective function coefficient is the same as the low end of the range of optimality for $x_{s}$.

Sometimes, in the sensitivity report we will have the final value of a variable is 0 and its reduced cost is also 0. When this occurs, it is another clue that we may have multiple optimal solutions. 

Can we force the solver to find the other corner point that is optimal? Yes. We can change the objective function into a constraint and then maximize $x_{p}$. (We are doing this only because we know that the alternate solution has a higher value for $x_{p}$.) Let's try it.

In [ ]:
# See the current model
m.display()

In [ ]:
# Turn the objective function into a constraint
m.addConstr(3.75*x_s + 5*x_p == 3000, name='set_obj_value_output')

# Change the objectiv function
m.setObjective(x_p)

m.update()
m.display()

In [ ]:
# solve
m.optimize()

# get the results
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

In [ ]:
sa_vars(m.getVars())

### Multiple Optimal Solutions

Let's consider the model where space rays bring in \\$9 of profit and phasers bring in \\$4.50 of profit. I will claim that this model also has multiple optimal solutions. Let's adjust our model and see if we can find multiple optimal solutions to it.

In [ ]:
# See current model
m.display()

In [ ]:
# Need to delete the 'set_obj_value_output' constraint
c = m.getConstrByName('set_obj_value_output')
if c:  # Check if the constraint exists
    m.remove(c)
    
# and change the objective function
m.setObjective(9 * x_s + 4.5 * x_p)
m.update()
m.display()

In [ ]:
# solve
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

In [ ]:
# Get the reduced cost and range of optimality for each variable
sa_vars(m.getVars())

In [ ]:
# get shadow prices and range of feasibility
sa_constrs(m.getConstrs())

#### How to Find Other Optimal Solutions

That answer only showed a single solution. We can ask Gurobi to find multiple solutions and then look at those.

In [ ]:
# do a systematic search for the k-best solutions
m.setParam(GRB.Param.PoolSearchMode, 2)

# Update the model
m.update()

# Solve
m.optimize()

# Get the status
status = m.Status
if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
    print('The model cannot be solved because it is infeasible or unbounded')

if status != GRB.OPTIMAL:
    print('Optimization was stopped with status ' + str(status))

if status == GRB.OPTIMAL:
    print(f'Optimization found an OPTIMAL answer with the status of {status}')

# Print number of solutions stored
nSolutions = m.SolCount
print('Number of solutions found: ' + str(nSolutions))

#### Why Only 1 Solution?

When the variables are continuous and you have multiple optimal solutions, you will in fact have an **infinite** number of optimal solutions. Therefore, the solver will not try to search for more than one. Which one did it find?

In [ ]:
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

#### Another Possible Approach?

Because the optimal solution found above is integer, you could try converting both variables to integer instead of continuous and re-solving the updated model. When dealing with integer or binary variables, Gurobi will try to find other optimal solutions.

In [ ]:
# change the variable types to integer
for var in m.getVars():
    print(f'Variable {var.VarName} is currently of type {var.VTYPE}')
    var.setAttr(GRB.Attr.VType, GRB.INTEGER)

# update the model for changes to take effect
m.update()
print('After setting the variables to integer ...')
for var in m.getVars():
    print(f'Variable {var.VarName} is currently of type {var.VTYPE}')

In [ ]:
# see current model
m.display()

In [ ]:
# Solve
m.optimize()

# Print number of solutions stored
nSolutions = m.SolCount
print(f'Number of solutions found: {nSolutions}')

In [ ]:
# Print objective value and values of the decision varaibles of solutions
for soln_num in range(m.getAttr(GRB.Attr.SolCount)):
    m.Params.SolutionNumber = soln_num
    print(f'solution {soln_num} has obj fn value of {m.PoolObjVal}')
    for v in m.getVars():
        # Need to use v.Xn to the value of the variable v for this solution
        print(f'   {v.VarName} = {v.Xn}')
    

### Success!

When using integer variables, we can get multiple optimal solutions with Gurobi. Note that within a solution you need to use **`v.Xn`**, where `v` is the variable, to get the values of the decision variables. 

**&copy; 2024 - Present: Matthew D. Dean, Ph.D.   
Clinical Professor of Business Analytics at William \& Mary.**